In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [2]:
mat = torch.randn(10, 10)

In [3]:
mat

tensor([[-1.4165, -0.8197,  0.5840, -0.5710,  1.1775, -2.6520, -1.8013, -0.0610,
         -0.0790, -0.6620],
        [ 0.1750, -2.3289,  0.6482,  0.0667, -1.2433,  0.4419,  0.0087,  1.6919,
         -0.4607,  1.5415],
        [-1.6358, -0.3130, -0.2227, -0.8180,  0.1609,  0.6328, -1.2006, -0.4555,
          0.2691, -1.8366],
        [ 0.4538, -0.1072,  1.4451,  0.7337,  1.0028,  1.9876, -0.4316,  0.2059,
          0.2663,  1.5349],
        [-0.6244,  0.1345,  1.0451, -0.3463, -0.9997,  0.7693, -2.0890, -1.8439,
          1.2456,  0.6228],
        [ 0.1525,  1.0256,  2.1814,  0.9934,  0.3796, -1.0118,  1.5973, -0.7096,
         -2.0966, -2.0054],
        [ 0.7749, -1.1679, -1.0441, -0.4934, -0.3041,  0.0481,  2.3030, -0.5135,
         -1.2164, -0.4723],
        [-1.2713, -1.2919, -0.1789,  1.5725, -1.8203, -0.2809, -0.6363,  0.5666,
         -1.0085, -0.1092],
        [-0.1462,  0.6494, -1.0594, -1.0009,  2.5404, -0.1336, -1.0451,  0.1042,
         -1.4020,  0.0568],
        [ 1.5819,  

In [6]:
data_iter = torch.utils.data.DataLoader(mat, batch_size=5, shuffle=True)